In [1]:
import cv2
import numpy as np
from scipy.spatial.distance import cosine
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.utils import load_img, img_to_array
import os
from tqdm.auto import tqdm

In [2]:
# 이미지 파일들이 저장된 폴더 경로
folder_path = r"C:\Users\NT550009\Desktop\detail_image_aggregated"

# 이미지 파일들의 목록 가져오기
image_files = os.listdir(folder_path)

# VGG16 모델 로드
# vgg_model = VGG16(weights='imagenet', include_top=False)

# Inception 모델 로드
inception_V3 = InceptionV3(weights='imagenet', include_top=False)

# 특성 벡터를 저장할 리스트 초기화
all_incetption_features = []
all_histograms = []
all_hue_features = []

# 이미지들에 대해 반복하여 특성 추출
for i in tqdm(range(0,10000)):
    try:
        # 이미지 파일 경로
        image_path = os.path.join(folder_path, image_files[i])
        
        # 이미지 로드 및 전처리
        img = load_img(image_path, target_size=(224, 224))
        x = img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)

        # VGG16 특성 추출
        # vgg_features = vgg_model.predict(x)
        # vgg_features = np.squeeze(vgg_features)
        # all_vgg_features.append(vgg_features)
        
        # InceptionV3 특성 추출
        inception_features = inception_V3.predict(x)
        inception_features = np.squeeze(inception_features)
        all_incetption_features.append(inception_features)

        # 색상 히스토그램 계산
        image_bgr = cv2.imread(image_path)
        image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
        histogram = cv2.calcHist([image_rgb], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
        all_histograms.append(histogram)

        # 색조(Hue) 특성 추출
        image_hsv = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2HSV)
        hue_channel = image_hsv[:, :, 0]
        hue_features = np.histogram(hue_channel, bins=8, range=(0, 180))[0]
        all_hue_features.append(hue_features)
    
    except cv2.error as e:
        if str(e) == '(-215:Assertion failed) !_src.empty() in function \'cv::cvtColor\'':
            # _src.empty() 조건이 만족되지 않은 경우에만 실행
            pass

  0%|          | 0/10000 [00:00<?, ?it/s]

1/1 [==============================] - 0s 74ms/step


In [ ]:
# 특성 벡터들을 저장
np.save('saved_inception_features.npy', np.array(all_incetption_features))
np.save('saved_histograms.npy', np.array(all_histograms))
np.save('saved_hue_features.npy', np.array(all_hue_features))

In [ ]:
# 이미지 파일 경로
query_path = r"C:\Users\NT550009\Desktop\recommendation\pants.jpg"

# VGG16 모델 로드
inception_V3 = InceptionV3(weights='imagenet', include_top=False)

# 이미지 로드 및 전처리
img = load_img(query_path, target_size=(224, 224))
x = img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

# VGG16 특성 추출
inception_features = inception_V3.predict(x)
inception_features = np.squeeze(inception_features)

# 색상 히스토그램 계산
image_bgr = cv2.imread(query_path)
image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
histogram = cv2.calcHist([image_rgb], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])

# 색조(Hue) 특성 추출
image_hsv = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2HSV)
hue_channel = image_hsv[:, :, 0]
hue_features = np.histogram(hue_channel, bins=8, range=(0, 180))[0]

# 저장된 특성 벡터 로드
saved_inception_features = np.load('saved_inception_features.npy')
saved_histograms = np.load('saved_histograms.npy')
saved_hue_features = np.load('saved_hue_features.npy')

# 유사도 측정을 위한 비교
inception_similarities = []
histogram_similarities = []
hue_similarities = []

for inception_feature in saved_inception_features:
    inception_similarity = 1 - cosine(inception_feature.flatten(), inception_features.flatten())
    inception_similarities.append(inception_similarity)

for saved_histogram in saved_histograms:
    histogram_similarity = cv2.compareHist(saved_histogram, histogram, cv2.HISTCMP_CORREL)
    histogram_similarities.append(histogram_similarity)

for saved_hue_feature in saved_hue_features:
    hue_similarity = 1 - cosine(saved_hue_feature, hue_features)
    hue_similarities.append(hue_similarity)

inception_similarities_array = np.array(inception_similarities)
# 종합 유사도 계산
weights = [5, 3, 2]  # 가중치 설정 average의 weights에는 소수 비율 안됨
# weights = np.array([5, 3, 2], dtype=np.float32) 
similarities = np.average([(inception_similarities[:5465]), np.array(histogram_similarities), np.array(hue_similarities)], axis=0, weights=weights)

# 유사도가 가장 높은 상위 N개의 인덱스 가져오기
top_indices = np.argsort(similarities)[::-1][:30]

# 유사한 이미지 파일 목록 가져오기
similar_image_files = [image_files[i] for i in top_indices]

In [ ]:
print("Most similar images:")
for image_file in similarities:
    print(image_file)

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

# ...

# Similar images to display
similar_images = similar_image_files[:30]

# Create a 6 by 5 subplot grid
fig, axs = plt.subplots(5, 6, figsize=(12, 12))

# Iterate over the similar images and display them
for i in range(len(similar_images)):
    image_path = r"C:\Users\NT550009\Desktop\detail_image_aggregated" + '/' + similar_images[i]
    # Load the image
    image = Image.open(image_path)

    # Calculate the subplot position
    row = i // 6
    col = i % 6

    # Display the image in the corresponding subplot
    axs[row, col].imshow(image)
    axs[row, col].axis('off')

# Adjust the spacing between subplots
plt.tight_layout()

# Show the plot
plt.show()